In [ ]:
import numpy as np
from ultralytics import YOLO
import cv2
import numpy as np
from screen_grab import ScreenCapture
import supervision as sv

# Object Detection 
In the code below we use a yolo11 model to detect all cars, busses, and trucks in the game frames. We set the confidence threshold to 0.5 and the NMS IoU threshold to 0.7. We then add bounding boxes to the image as well as a label noting the name of the identified class and the confidence. This code finishes by capturing the game and letting the model loose on those images.

In [ ]:
# Load the YOLO model
model = YOLO('yolo11s.pt')

def detect_objects(frame):
    # Convert PIL(Python Imaging Library) image to OpenCV format
    frame = np.array(frame)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Perform YOLO prediction
    results = model(frame)

    # Parse detections
    detections = sv.Detections.from_ultralytics(results[0]).with_nms(threshold=0.7)

    selected_classes = [2, 5, 7] #detect only cars, busses and trucks
    detections = detections[np.isin(detections.class_id, selected_classes)]
    detections = detections[detections.confidence > 0.5] 
    
    # Give object a bounding box and a label
    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()

    labels = [
        f"{class_name} {confidence:.2f}"
        for class_name, confidence
        in zip(detections['class_name'], detections.confidence)
    ]

    annotated_image = bounding_box_annotator.annotate(
        scene=frame, detections=detections)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections, labels=labels)
    
    return annotated_image

# Capture screen from the game window and run YOLO detection
game_title = "Forza Horizon 4"
while True:
    screen_capture = ScreenCapture()

    if screen_capture.game_area:
        screen = screen_capture.capture_frame()
        detected_frame = detect_objects(screen)

        # Display the detection
        cv2.imshow("Game Object Detection", detected_frame)


    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


pygame 2.6.0 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html



RuntimeError: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 4, 384, 640] to have 3 channels, but got 4 channels instead

## Custom Yolo Model
The below code is used to train a custom model that can recognize trees and houses in the game.

In [ ]:
!yolo task=detect mode=train model=yolo11s.pt data=CVC_v1i_yolov11/data.yaml epochs=5
